In [1]:
# | label: setup
# | echo: false

from pathlib import Path

import duckdb
import pandas as pd

In [2]:
# | label: data_source
# | echo: false

DATA_URL = "https://hbiostat.org/data/repo/titanic3.csv"  # hopefully this is a "definitive" source

#### DuckDB setup

In [12]:
def load_data_duckdb(db_name=None, table_name="raw_titanic", data_url=DATA_URL):
    load_data_sql = f"CREATE OR REPLACE TABLE {table_name} AS SELECT * FROM read_csv_auto('{data_url}')"
    if db_name is None:
        con = duckdb.connect()
    else:
        con = duckdb.connect(db_name)
    con.sql(load_data_sql)
    print(load_data_sql)
    return con, table_name

In [13]:
def load_sql_query(file_path):
    """Load the contents of a .sql file if it exists.

    Args:
    file_path (str): The path to the .sql file.

    Returns:
    str: The contents of the .sql file ready to be processed by a duckdb connection.
    """
    file = Path(file_path)
    if not file.is_file() or file.suffix != '.sql':
        raise FileNotFoundError("The file does not exist or is not a .sql file.")
    sql = file.read_text()
    print(sql)
    return sql


In [14]:
def create_view(con, view_name, sql):
    con.sql(f"CREATE VIEW {view_name} AS {sql}")

In [15]:
DB_NAME = "../try-evidence-template/sources/titanic/titanic.ddb"

Path(DB_NAME).unlink(missing_ok=True)
Path(f"{DB_NAME}.wal").unlink(missing_ok=True)

In [16]:
con, table_name = load_data_duckdb(db_name=DB_NAME, data_url=DATA_URL)  # this CSV has home.dest renamed to home_dest

CREATE OR REPLACE TABLE raw_titanic AS SELECT * FROM read_csv_auto('https://hbiostat.org/data/repo/titanic3.csv')


: 

In [8]:
con.sql("SHOW TABLES;")

┌─────────────┐
│    name     │
│   varchar   │
├─────────────┤
│ raw_titanic │
└─────────────┘

In [9]:
con.table(table_name)

┌────────┬──────────┬──────────────────────┬─────────┬────────┬───┬──────────┬─────────┬───────┬──────────────────────┐
│ pclass │ survived │         name         │   sex   │  age   │ … │ embarked │  boat   │ body  │      home.dest       │
│ int64  │  int64   │       varchar        │ varchar │ double │   │ varchar  │ varchar │ int64 │       varchar        │
├────────┼──────────┼──────────────────────┼─────────┼────────┼───┼──────────┼─────────┼───────┼──────────────────────┤
│      1 │        1 │ Allen, Miss. Elisa…  │ female  │   29.0 │ … │ S        │ 2       │  NULL │ St Louis, MO         │
│      1 │        1 │ Allison, Master. H…  │ male    │   0.92 │ … │ S        │ 11      │  NULL │ Montreal, PQ / Che…  │
│      1 │        0 │ Allison, Miss. Hel…  │ female  │    2.0 │ … │ S        │ NULL    │  NULL │ Montreal, PQ / Che…  │
│      1 │        0 │ Allison, Mr. Hudso…  │ male    │   30.0 │ … │ S        │ NULL    │   135 │ Montreal, PQ / Che…  │
│      1 │        0 │ Allison, Mrs. Huds

In [10]:
con.sql(f"SELECT * FROM {table_name}").df()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,S,None,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",female,14.50,1,0,2665,14.4542,None,C,None,328.0,None
1305,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,None,C,None,NaN,None
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.50,0,0,2656,7.2250,None,C,None,304.0,None
1307,3,0,"Zakarian, Mr. Ortin",male,27.00,0,0,2670,7.2250,None,C,None,NaN,None


In [11]:
con.close()